Treure text del fitxer

In [ ]:
import torch

with open('data/quijote.txt', 'r') as file:
    text = file.read()
print(text)

Separar el text en chunks per a facilitar l'embedding

In [ ]:
import tiktoken

def split_text(text, chunk_size=400, overlap=50):
    tokenizer = tiktoken.get_encoding("cl100k_base")  # Usar el tokenizer adecuado para tu modelo
    tokens = tokenizer.encode(text)
    
    chunks = []
    for i in range(0, len(tokens), chunk_size - overlap):
        chunk = tokens[i : i + chunk_size]
        chunks.append(tokenizer.decode(chunk))
    
    return chunks

chunks = split_text(text)


In [ ]:
from sentence_transformers import SentenceTransformer

device = 'cuda' if torch.cuda.is_available() else 'cpu'

def create_embeddings(chunks):
    model = SentenceTransformer('all-MiniLM-L6-v2')

    model.to(device)

    embeddings = model.encode(chunks, convert_to_tensor=True).to('cpu')
    return embeddings

embeddings = create_embeddings(chunks)

In [ ]:
from pinecone import Pinecone, ServerlessSpec

# Inicializa Pinecone
pc = Pinecone(api_key="pcsk_3EQD3c_TSypvXiqmCEavKDM7voRpMHVzd9FXnyAAu8LVCerLNtA69tUziFNVfFDH77WDTs")

# Crea un índice si no existe
index_name = "quijote-index"
dimension = embeddings.shape[1]
print(dimension)
# if index_name not in pc.list_indexes():
#     pc.create_index(index_name, dimension=dimension, spec=ServerlessSpec(cloud="aws", region="us-east-1"), metric="cosine")

In [ ]:

# Conecta al índice
index = pc.Index(index_name)

# def upload_to_pinecone(chunks, embeddings, batch_size=100):
#     vectors = [(str(i), embedding.tolist(), {"text": chunk}) for i, (chunk, embedding) in enumerate(zip(chunks, embeddings))]
#     for i in range(0, len(vectors), batch_size):
#         batch = vectors[i:i+batch_size]
#         index.upsert(vectors=batch)

# upload_to_pinecone(chunks, embeddings)

In [ ]:
def embed_query(query):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    model.to(device)
    embedded_query = model.encode(query, convert_to_tensor=True).to('cpu')
    return embedded_query

def query_index(embedded_query, index):
    query_response = index.query(
        vector=embedded_query, top_k=2, include_metadata=True
    )
    return query_response

query = embed_query("Cómo se llama el ayudante de Don Quijote de la Mancha?")


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("projecte-aina/FlorRAG")
model = AutoModelForCausalLM.from_pretrained("projecte-aina/FlorRAG")

instruction = "Cómo se llama el ayudante de Don Quijote de la Mancha?"
context = query_index(query.tolist(), index)

def givePrediction(instruction, context):
    question = f"### Instruction\n{{instruction}}\n### Context\n{{context}}\n### Answer\n"
    response = model(question, context)
    answer = response.split("###")[-1][8:-1]
    return answer

answer = givePrediction(instruction, context)

print(answer)

In [ ]:
id_answer = int(context["matches"][0]["id"])
print(chunks[id_answer])